In [1]:
import numpy as np
import os
import pandas as pd
from PIL import Image
from PIL import ImageOps , ImageFilter
import tensorflow as tf
import matplotlib.pyplot as plt
import cv2

In [2]:
# base_paths = ['COVID-19_Radiography_Dataset','Pneumonia']
base_paths = ['Lungs X Ray']
IMAGE_WIDTH = 200
IMAGE_HEIGHT = 200

batch_size = 16
# batch_size = 5

IMAGE_SIZE = (IMAGE_WIDTH, IMAGE_HEIGHT)
IMAGE_CHANNELS = 3

In [3]:
directories = ['Normal', 'COVID', 'Viral Pneumonia']
# categories = []
    
# Create a list to store file information
data = []

In [5]:
# Iterate through the directories and collect file information
for i, category in enumerate(directories):
    for base_path in base_paths:
        category_path = os.path.join(base_path, category, "images")
        if os.path.exists(category_path):
            for filename in os.listdir(category_path):
                data.append({'filename': os.path.join(base_path,category, "images", filename), 'category': i})

# Create a DataFrame from the collected data
df = pd.DataFrame(data)

# Display the first few rows of the DataFrame
print(df.head())

                                      filename  category
0  Lungs X Ray\Normal\images\IM-0214-0001.jpeg         0
1  Lungs X Ray\Normal\images\IM-0215-0001.jpeg         0
2  Lungs X Ray\Normal\images\IM-0216-0001.jpeg         0
3  Lungs X Ray\Normal\images\IM-0217-0001.jpeg         0
4  Lungs X Ray\Normal\images\IM-0218-0001.jpeg         0


In [6]:
df.tail()

,filename,category
93,Lungs X Ray\COVID\images\covid (8).jpg,1
94,Lungs X Ray\COVID\images\covid (8).png,1
95,Lungs X Ray\COVID\images\covid (9).jpeg,1
96,Lungs X Ray\COVID\images\covid (9).jpg,1
97,Lungs X Ray\COVID\images\covid (9).png,1


In [7]:
df['category'].value_counts()

1    70
0    28
Name: category, dtype: int64

In [8]:
Covid_df = df[df['category'] == 1]
Covid_df.head()

,filename,category
28,Lungs X Ray\COVID\images\covid (1).jpeg,1
29,Lungs X Ray\COVID\images\covid (1).jpg,1
30,Lungs X Ray\COVID\images\covid (1).png,1
31,Lungs X Ray\COVID\images\covid (10).jpeg,1
32,Lungs X Ray\COVID\images\covid (10).jpg,1


In [9]:
# Define the CLAHE function
def apply_clahe(image):
    # Convert the image to LAB color space
    lab = cv2.cvtColor(image, cv2.COLOR_RGB2LAB)

    # Split the LAB image into channels
    l, a, b = cv2.split(lab)

    # Apply CLAHE to the L-channel
    clahe = cv2.createCLAHE(clipLimit=3.0, tileGridSize=(8, 8))
    cl = clahe.apply(l)

    # Merge the CLAHE-enhanced L-channel with the original A and B channels
    limg = cv2.merge((cl, a, b))

    # Convert LAB back to RGB
    clahe_image = cv2.cvtColor(limg, cv2.COLOR_LAB2RGB)

    return clahe_image

In [9]:
def contrast_stretching(image):
    # Convert image to NumPy array
    img_array = tf.keras.preprocessing.image.img_to_array(image)
    
    # Normalize pixel values to the range [0, 1]
    # img_array = img_array / 255.0
    
    # Apply contrast stretching
    img_array = ((img_array - img_array.min()) / (img_array.max() - img_array.min()))*255.0
    
    # Convert back to image
    stretched_image = tf.keras.preprocessing.image.array_to_img(img_array)
    
    return stretched_image

In [10]:
def histogram_equalization(image):
    img_array = tf.keras.preprocessing.image.img_to_array(image)
    img_array = (img_array - img_array.min()) / (img_array.max() - img_array.min())  # Normalize pixel values
    img_array = (img_array * 255).astype(np.uint8)
    
    img_pil = Image.fromarray(img_array.astype('uint8'))  
    img_equalized = ImageOps.equalize(img_pil)
    
    return img_equalized

In [11]:
#HE with Median Filter
def he_Median(image, median_kernel_size=3):
    # Convert image to NumPy array
    img_array = tf.keras.preprocessing.image.img_to_array(image)
    
    # Normalize pixel values to the range [0, 1]
    img_array = (img_array - img_array.min()) / (img_array.max() - img_array.min())  
    
    # Scale to the range [0, 255]
    img_array = (img_array * 255).astype(np.uint8)
    
    # Convert back to image
    img_pil = Image.fromarray(img_array.astype('uint8'))  
    
    # Perform histogram equalization
    img_equalized = ImageOps.equalize(img_pil)
    
    # Apply median filter
    img_equalized_median = img_equalized.filter(ImageFilter.MedianFilter(size=median_kernel_size))
    
    return img_equalized_median

In [12]:
#Contrast Streching with Median Filter
def contrast_stretching_and_median(image, median_kernel_size=3):
    # Convert image to NumPy array
    img_array = tf.keras.preprocessing.image.img_to_array(image)
    
    
    img_array = ((img_array - img_array.min()) / (img_array.max() - img_array.min())) * 255.0
    
    # Apply contrast stretching
    img_array_stretched = img_array
    
    # Apply median filter
    img_array_median = cv2.medianBlur(img_array_stretched, median_kernel_size)
    
    # Combine the stretched and median filtered images (you can adjust this combination as needed)
    # img_array_combined = (img_array_stretched + img_array_median) / 2.0
    img_array_combined = img_array_median
    
    # Convert back to image
    combined_image = tf.keras.preprocessing.image.array_to_img(img_array_combined)
    
    return combined_image


In [12]:
output_dir1 = "CLAHE"

In [10]:
# CLAHE Preprocessing and Save
def clahe_prep(idf,path):
    for index, row in idf.iterrows():
        file = row['filename']  # Assuming the file name is in a column named 'filename'
        
        subdirectories = file.split(os.path.sep)  # Split the file path using the platform-specific separator
        filename = subdirectories[-1]
        subdirectories = subdirectories[:-1]  # Exclude the filename itself
        output_subdir1 = os.path.join(path, *subdirectories)
        
        # Create the subdirectories if they don't exist
        if not os.path.exists(output_subdir1):
            os.makedirs(output_subdir1)
        # print(file)
        # input_path = os.path.join(base_path, file)
        
        # Load the image using OpenCV
        image = cv2.imread(file)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)  # Convert BGR to RGB

        # Apply CLAHE preprocessing
        clahe_image = apply_clahe(image)
        
        # Save the processed image to the output directory
        output_path1 = os.path.join(output_subdir1, f"CLAHE-{filename}")
        cv2.imwrite(output_path1, cv2.cvtColor(clahe_image, cv2.COLOR_RGB2BGR))

In [15]:
#CLAHE + MEDIAN
def apply_clahe_and_median(image, median_kernel_size=3):
    # Convert the image to LAB color space
    lab = cv2.cvtColor(image, cv2.COLOR_RGB2LAB)

    # Split the LAB image into channels
    l, a, b = cv2.split(lab)

    # Apply CLAHE to the L-channel
    clahe = cv2.createCLAHE(clipLimit=3.0, tileGridSize=(8, 8))
    cl = clahe.apply(l)

    # Merge the CLAHE-enhanced L-channel with the original A and B channels
    limg = cv2.merge((cl, a, b))

    # Convert LAB back to RGB
    clahe_image = cv2.cvtColor(limg, cv2.COLOR_LAB2RGB)
    
    # Apply median filter
    clahe_median = cv2.medianBlur(clahe_image, median_kernel_size)

    return clahe_median

In [32]:
# CLAHE+MEDIAN Preprocessing and Save
def clahe_median_prep(idf,path):
    for index, row in idf.iterrows():
        file = row['filename']  # Assuming the file name is in a column named 'filename'
        
        subdirectories = file.split(os.path.sep)  # Split the file path using the platform-specific separator
        filename = subdirectories[-1]
        subdirectories = subdirectories[:-1]  # Exclude the filename itself
        output_subdir1 = os.path.join(path, *subdirectories)
        
        # Create the subdirectories if they don't exist
        if not os.path.exists(output_subdir1):
            os.makedirs(output_subdir1)

        # input_path = os.path.join(base_path, file)
        
        # Load the image using OpenCV
        image = cv2.imread(file)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)  # Convert BGR to RGB

        # Apply CLAHE preprocessing
        clahe_image = apply_clahe_and_median(image)
        
        # Save the processed image to the output directory
        output_path1 = os.path.join(output_subdir1, f"CLAHE+MEDIAN-{filename}")
        cv2.imwrite(output_path1, cv2.cvtColor(clahe_image, cv2.COLOR_RGB2BGR))

In [37]:
#Histogram Preprocessing and Save
def histogramEQ(idf,path):
    for index, row in idf.iterrows():
        file = row['filename']  # Assuming the file name is in a column named 'filename'
        
        subdirectories = file.split(os.path.sep)  # Split the file path using the platform-specific separator
        filename = subdirectories[-1]
        subdirectories = subdirectories[:-1]  # Exclude the filename itself
        output_subdir1 = os.path.join(path, *subdirectories)
        
        # Create the subdirectories if they don't exist
        if not os.path.exists(output_subdir1):
            os.makedirs(output_subdir1)

        # input_path = os.path.join(base_path, file)
        
        # Load the image using tf
        image = tf.keras.preprocessing.image.load_img(file)
        # image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)  # Convert BGR to RGB

        # Apply CLAHE preprocessing
        histo_image = histogram_equalization(image)
        
        # Save the processed image to the output directory
        output_path1 = os.path.join(output_subdir1, f"HE-{filename}")
        histo_image.save(output_path1)

In [18]:
#Histogram Preprocessing + Median Filter and Save
def histogramEQ_M(idf,path):
    for index, row in idf.iterrows():
        file = row['filename']  # Assuming the file name is in a column named 'filename'
        
        subdirectories = file.split(os.path.sep)  # Split the file path using the platform-specific separator
        filename = subdirectories[-1]
        subdirectories = subdirectories[:-1]  # Exclude the filename itself
        output_subdir1 = os.path.join(path, *subdirectories)
        
        # Create the subdirectories if they don't exist
        if not os.path.exists(output_subdir1):
            os.makedirs(output_subdir1)

        input_path = os.path.join(base_path, file)
        
        # Load the image using tf
        image = tf.keras.preprocessing.image.load_img(input_path)
        # image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)  # Convert BGR to RGB

        # Apply CLAHE preprocessing
        histo_image = he_Median(image)
        
        # Save the processed image to the output directory
        output_path1 = os.path.join(output_subdir1, f"HE+M-{filename}")
        histo_image.save(output_path1)

In [19]:
output_CS = 'Constrast_Streched'

In [20]:
#Contrast Streching Save
def constrast_ST(idf,path):
    for index, row in idf.iterrows():
        file = row['filename']  # Assuming the file name is in a column named 'filename'


        subdirectories = file.split(os.path.sep)  # Split the file path using the platform-specific separator
        filename = subdirectories[-1]
        subdirectories = subdirectories[:-1]  # Exclude the filename itself
        output_subdir1 = os.path.join(path, *subdirectories)

        if not os.path.exists(output_subdir1):
            os.makedirs(output_subdir1)


        input_path = os.path.join(base_path, file)
        
        # Load the image using TensorFlow
        image = tf.keras.preprocessing.image.load_img(input_path)
        
        # Apply contrast stretching
        stretched_image_cov = contrast_stretching(image)
        # equalized_image = histogram_equalization(image)
        
        # Save the processed image to the output directory
    
        output_path1 = os.path.join(output_subdir1,f"CS-{filename}")
        # output_path2 = os.path.join(output_subdir2,f"COVID-EQ-{index}.png")
        stretched_image_cov.save(output_path1)
    # equalized_image.save(output_path2)

In [30]:
#Contrast Streching With Median Filter
def constrast_ST_Median(idf,path):
    for index, row in idf.iterrows():
        file = row['filename']  # Assuming the file name is in a column named 'filename'


        subdirectories = file.split(os.path.sep)  # Split the file path using the platform-specific separator
        filename = subdirectories[-1]
        subdirectories = subdirectories[:-1]  # Exclude the filename itself
        output_subdir1 = os.path.join(path, *subdirectories)

        if not os.path.exists(output_subdir1):
            os.makedirs(output_subdir1)


        # input_path = os.path.join(base_path, file)
        
        # Load the image using TensorFlow
        image = tf.keras.preprocessing.image.load_img(file)
        
        # Apply contrast stretching with Median Filter
        stretched_image_cov = contrast_stretching_and_median(image)
        
        # equalized_image = histogram_equalization(image)
        
        # Save the processed image to the output directory
    
        output_path1 = os.path.join(output_subdir1,f"CS+MEDIAN-{filename}")
        # output_path2 = os.path.join(output_subdir2,f"COVID-EQ-{index}.png")
        stretched_image_cov.save(output_path1)
    # equalized_image.save(output_path2)

In [31]:
constrast_ST_Median(Covid_df,'CS+MEDIAN')
# constrast_ST_Median(pneum_df,'CS+MEDIAN')

In [40]:
# clahe_median_prep(Covid_df,'CL+MEDIAN')
clahe_median_prep(pneum_df,'CL+MEDIAN')

In [38]:
histogramEQ(pneum_df,'HE')

In [79]:
histogramEQ_M(pneum_df,'HE+M')

In [37]:
constrast_ST(Covid_df,output_CS)

In [36]:
pneum_df =df[df['category'] == 2]
pneum_df.head()

,filename,category
15149,COVID-19_Radiography_Dataset\Viral Pneumonia\i...,2
15150,COVID-19_Radiography_Dataset\Viral Pneumonia\i...,2
15151,COVID-19_Radiography_Dataset\Viral Pneumonia\i...,2
15152,COVID-19_Radiography_Dataset\Viral Pneumonia\i...,2
15153,COVID-19_Radiography_Dataset\Viral Pneumonia\i...,2


In [13]:
clahe_prep(Covid_df,output_dir1)
# clahe_prep(pneum_df,output_dir1)

In [32]:
constrast_ST(pneum_df,output_CS)